#### Data
Hiking: https://www.phoenixopendata.com/dataset/hiking-trail-usage/resource/aa4e2a08-c0ad-4fc4-bee9-44c2d85a58fa
<br>
Weather: https://www.noaa.gov/

In [11]:
import pandas as pd
import numpy as np
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 500)
pd.set_option('display.max_columns', 500)

### Trails

In [12]:
df = pd.read_csv('../data/trails.csv')
df.columns = [x.lower() for x in df.columns]
df.head()

,date,site,count
0,1/1/2019,E - Camelback - Cholla Trail,NaN
1,1/1/2019,E - Camelback - Echo Canyon Trail,1259.0
2,1/1/2019,E - Dreamy Draw Park - Bike Path - Restroom Area,588.0
3,1/1/2019,E - Dreamy Draw Park - Trail 100,181.0
4,1/1/2019,E - Papago - 5K Fitness,264.0


In [13]:
# fill na with 0
df['count'] = df['count'].fillna(0)

# Separate days, months and years
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].apply(lambda x: x.month)
df['day'] = df['date'].apply(lambda x: x.day)
df['year'] = df['date'].apply(lambda x: x.year)
df['week'] = df['day'].apply(lambda x: (x-1)//7+1)
df['weekday'] = df['date'].apply(lambda x: x.weekday())

# Add holidays
cal = calendar()
holidays = cal.holidays(start=df['date'].min(), end=df['date'].max())
df['is_holiday'] = df['date'].isin(holidays).astype(int)

# Split address from trail
def split_section(x):
    a = x.split('-')
    if 'South Mountain' in x:
        return pd.Series({'location': a[0].strip(),'trail': a[1].strip()})
    elif len(a) == 3:
        return pd.Series({'location': '-'.join(a[:2]).strip(), 'trail': '-'.join(a[2:]).strip()})
    elif len(a) >= 4:
        return pd.Series({'location': '-'.join(a[:3]).strip(), 'trail': '-'.join(a[3:]).strip()})
    else:
        return pd.Series({'location': x, 'trail': x})
    
df[['location', 'trail']] = df['site'].apply(split_section)

# Get seasons from dates
conditions = [
    (df['date'].dt.month >= 12) | (df['date'].dt.month <= 2),  # Winter: December to February
    (df['date'].dt.month >= 3) & (df['date'].dt.month <= 5),   # Spring: March to May
    (df['date'].dt.month >= 6) & (df['date'].dt.month <= 8),   # Summer: June to August
    (df['date'].dt.month >= 9) & (df['date'].dt.month <= 11)   # Fall: September to November
]

# Define the corresponding season labels
seasons = ['winter', 'spring', 'summer', 'fall']

# Apply the conditions and assign the corresponding season label
df['season'] = pd.Series(np.select(conditions, seasons, default='winter'))

# Create a function to handle leap years
def is_leap_year(year):
    return (year % 4 == 0 and year % 100 != 0) or year % 400 == 0

# Calculate previous week, month, and year dates
df['prev_week_date'] = df['date'] - pd.Timedelta(days=7)
df['prev_month_date'] = df['date'] - pd.offsets.MonthBegin(1)
df['prev_year_date'] = df['date'] - pd.offsets.YearBegin(1)

# Adjust previous month for leap years (using is_leap_year)
df.loc[df['prev_month_date'].dt.day == 29, 'prev_month_date'] = df.apply(lambda row: row['prev_month_date'] - pd.Timedelta(days=1) if is_leap_year(row['prev_month_date'].year) else row['prev_month_date'], axis=1)

# Calculate total counts for previous week, month, and year (with explicit reindexing)
df['total_previous_week'] = df.groupby('site',group_keys=False)['count'].transform(lambda x: x.shift(7).rolling(7).sum().reindex(x.index, fill_value=0))
df['total_previous_month'] = df.groupby('site',group_keys=False)['count'].transform(lambda x: x.shift(1).rolling(30).sum().reindex(x.index, fill_value=0))
df['total_previous_year'] = df.groupby('site',group_keys=False)['count'].transform(lambda x: x.shift(1).rolling(365).sum().reindex(x.index, fill_value=0))

# Get amounts for the same day last week, month, and year (with explicit reindexing)
df['amount_same_day_last_week'] = df.groupby(['site', df['date'].dt.weekday],group_keys=False)['count'].apply(lambda x: x.shift(7).reindex(x.index, fill_value=0))
df['amount_same_day_last_month'] = df.groupby(['site', df['date'].dt.day],group_keys=False)['count'].apply(lambda x: x.shift(1).reindex(x.index, fill_value=0))
df['amount_same_day_last_year'] = df.groupby(['site', df['date'].dt.dayofyear],group_keys=False)['count'].apply(lambda x: x.shift(1).reindex(x.index, fill_value=0))

# Handle missing values (e.g., first week/month/year)
df.fillna(0, inplace=True)

In [14]:
df.head()

,date,site,count,month,day,year,week,weekday,is_holiday,location,trail,season,prev_week_date,prev_month_date,prev_year_date,total_previous_week,total_previous_month,total_previous_year,amount_same_day_last_week,amount_same_day_last_month,amount_same_day_last_year
0,2019-01-01,E - Camelback - Cholla Trail,0.0,1,1,2019,1,1,1,E - Camelback,Cholla Trail,winter,2018-12-25,2018-12-01,2018-01-01,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-01-01,E - Camelback - Echo Canyon Trail,1259.0,1,1,2019,1,1,1,E - Camelback,Echo Canyon Trail,winter,2018-12-25,2018-12-01,2018-01-01,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-01-01,E - Dreamy Draw Park - Bike Path - Restroom Area,588.0,1,1,2019,1,1,1,E - Dreamy Draw Park - Bike Path,Restroom Area,winter,2018-12-25,2018-12-01,2018-01-01,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-01-01,E - Dreamy Draw Park - Trail 100,181.0,1,1,2019,1,1,1,E - Dreamy Draw Park,Trail 100,winter,2018-12-25,2018-12-01,2018-01-01,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-01-01,E - Papago - 5K Fitness,264.0,1,1,2019,1,1,1,E - Papago,5K Fitness,winter,2018-12-25,2018-12-01,2018-01-01,0.0,0.0,0.0,0.0,0.0,0.0


### Weather

In [15]:
# WEATHER DATA INFO
# Units: Standard
# Stations: PHOENIX AIRPORT, AZ US (GHCND:USW00023183)

# Custom Flag(s)	Station Name, Geographic Location
# WSF2 - Fastest 2-minute wind speed
# WSF5 - Fastest 5-second wind speed
# SNOW - Snowfall
# WT03 - Thunder
# PRCP - Precipitation
# WT07 - Dust, volcanic ash, blowing dust, blowing sand, or blowing obstruction
# WT08 - Smoke or haze
# SNWD - Snow depth
# WDF2 - Direction of fastest 2-minute wind
# AWND - Average wind speed
# WDF5 - Direction of fastest 5-second wind
# WT10 - Tornado, waterspout, or funnel cloud"
# PGTM - Peak gust time
# WT01 - Fog, ice fog, or freezing fog (may include heavy fog)
# TMAX - Maximum temperature
# WT02 - Heavy fog or heaving freezing fog (not always distinguished from fog)
# TAVG - Average Temperature.
# TMIN - Minimum temperature

# cols needed: ['TMAX', 'TMIN', 'WDF2', 'WT07', 'WDF5', 'WSF2', 'WSF5, TAVG']

In [16]:
weather = pd.read_csv('../data/weather-data.csv')
weather.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT07,WT08,WT10
0,USW00023183,"PHOENIX AIRPORT, AZ US",33.4278,-112.00365,339.2,2019-01-01,3.58,NaN,0.00,NaN,NaN,43.0,51,38,10,10.0,12.1,18.1,1.0,NaN,NaN,NaN,NaN,NaN
1,USW00023183,"PHOENIX AIRPORT, AZ US",33.4278,-112.00365,339.2,2019-01-02,3.36,NaN,0.00,NaN,NaN,41.0,52,30,180,210.0,8.1,11.0,NaN,NaN,NaN,NaN,NaN,NaN
2,USW00023183,"PHOENIX AIRPORT, AZ US",33.4278,-112.00365,339.2,2019-01-03,3.58,NaN,0.00,NaN,NaN,43.0,57,33,90,110.0,10.1,15.0,NaN,NaN,NaN,NaN,NaN,NaN
3,USW00023183,"PHOENIX AIRPORT, AZ US",33.4278,-112.00365,339.2,2019-01-04,4.03,NaN,0.00,NaN,NaN,48.0,67,37,90,80.0,10.1,15.0,NaN,NaN,NaN,NaN,NaN,NaN
4,USW00023183,"PHOENIX AIRPORT, AZ US",33.4278,-112.00365,339.2,2019-01-05,6.04,NaN,0.07,NaN,NaN,54.0,67,45,160,160.0,15.0,21.9,1.0,NaN,NaN,NaN,NaN,NaN


In [17]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1826 entries, 0 to 1825
Data columns (total 24 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   STATION    1826 non-null   object 
 1   NAME       1826 non-null   object 
 2   LATITUDE   1826 non-null   float64
 3   LONGITUDE  1826 non-null   float64
 4   ELEVATION  1826 non-null   float64
 5   DATE       1826 non-null   object 
 6   AWND       1826 non-null   float64
 7   PGTM       8 non-null      float64
 8   PRCP       1826 non-null   float64
 9   SNOW       699 non-null    float64
 10  SNWD       610 non-null    float64
 11  TAVG       1821 non-null   float64
 12  TMAX       1826 non-null   int64  
 13  TMIN       1826 non-null   int64  
 14  WDF2       1826 non-null   int64  
 15  WDF5       1825 non-null   float64
 16  WSF2       1826 non-null   float64
 17  WSF5       1825 non-null   float64
 18  WT01       91 non-null     float64
 19  WT02       3 non-null      float64
 20  WT03    

In [18]:
weather_cols = ['DATE','TMAX', 'TMIN', 'TAVG', 'WDF2', 'WT07', 'WDF5', 'WSF2', 'WSF5']
weather = weather[weather_cols].fillna(0)
weather['DATE'] = pd.to_datetime(weather['DATE'])
weather.head()

,DATE,TMAX,TMIN,TAVG,WDF2,WT07,WDF5,WSF2,WSF5
0,2019-01-01,51,38,43.0,10,0.0,10.0,12.1,18.1
1,2019-01-02,52,30,41.0,180,0.0,210.0,8.1,11.0
2,2019-01-03,57,33,43.0,90,0.0,110.0,10.1,15.0
3,2019-01-04,67,37,48.0,90,0.0,80.0,10.1,15.0
4,2019-01-05,67,45,54.0,160,0.0,160.0,15.0,21.9


In [19]:
# Merge trail and weather and save file
df = df.merge(weather, left_on='date', right_on='DATE')
df.drop(columns=['DATE'], inplace=True)
df.to_csv('../data/trails-weather-clean.csv', encoding='utf-8')

In [20]:
df.head()

,date,site,count,month,day,year,week,weekday,is_holiday,location,trail,season,prev_week_date,prev_month_date,prev_year_date,total_previous_week,total_previous_month,total_previous_year,amount_same_day_last_week,amount_same_day_last_month,amount_same_day_last_year,TMAX,TMIN,TAVG,WDF2,WT07,WDF5,WSF2,WSF5
0,2019-01-01,E - Camelback - Cholla Trail,0.0,1,1,2019,1,1,1,E - Camelback,Cholla Trail,winter,2018-12-25,2018-12-01,2018-01-01,0.0,0.0,0.0,0.0,0.0,0.0,51,38,43.0,10,0.0,10.0,12.1,18.1
1,2019-01-01,E - Camelback - Echo Canyon Trail,1259.0,1,1,2019,1,1,1,E - Camelback,Echo Canyon Trail,winter,2018-12-25,2018-12-01,2018-01-01,0.0,0.0,0.0,0.0,0.0,0.0,51,38,43.0,10,0.0,10.0,12.1,18.1
2,2019-01-01,E - Dreamy Draw Park - Bike Path - Restroom Area,588.0,1,1,2019,1,1,1,E - Dreamy Draw Park - Bike Path,Restroom Area,winter,2018-12-25,2018-12-01,2018-01-01,0.0,0.0,0.0,0.0,0.0,0.0,51,38,43.0,10,0.0,10.0,12.1,18.1
3,2019-01-01,E - Dreamy Draw Park - Trail 100,181.0,1,1,2019,1,1,1,E - Dreamy Draw Park,Trail 100,winter,2018-12-25,2018-12-01,2018-01-01,0.0,0.0,0.0,0.0,0.0,0.0,51,38,43.0,10,0.0,10.0,12.1,18.1
4,2019-01-01,E - Papago - 5K Fitness,264.0,1,1,2019,1,1,1,E - Papago,5K Fitness,winter,2018-12-25,2018-12-01,2018-01-01,0.0,0.0,0.0,0.0,0.0,0.0,51,38,43.0,10,0.0,10.0,12.1,18.1


### Dashboard: <br>
https://public.tableau.com/app/profile/igor.rodrigues/viz/PhoenixHikingTrails/Trails